# Key Performance Indicators

In [1]:
#Import the libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yaml
import math

%matplotlib inline

In [2]:
#Load dataset
df = pd.read_csv("../data/clean/final_df.csv")
columns_to_drop = ['Unnamed: 0']
df = df.drop(columns=columns_to_drop)
df.head()

,client_id,tenure_yr,tenure_mnth,age,gender,num_acc,balance,calls,log_ons,variation,visitor_id,visit_id,process_step,date_time,date,time
0,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:51:13,2017-04-02,11:51:13
1,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:47:50,2017-04-02,11:47:50
2,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:46:45,2017-04-02,11:46:45
3,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,step_3,2017-04-02 11:23:08,2017-04-02,11:23:08
4,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,step_2,2017-04-02 11:22:24,2017-04-02,11:22:24


### Lets explore the three KPIs for the 'Test' group

In [3]:
#Filter for 'Test' group
test_group = df[df['variation'] == 'test']

#### Completion Rate

In [4]:
#Completion rate of the test group
completion_test = test_group[test_group['process_step'] == 'confirm']
completion_test_unique = completion_test['client_id'].nunique()
total_test = test_group['client_id'].nunique()
completion_rate_test = completion_test_unique / total_test
print(f"The completion rate for the Test group is: {completion_rate_test:.3f}")

The completion rate for the Test group is: 0.693


In [5]:
test_group['completion_rate'] = completion_rate_test
test_group.head(5)

/var/folders/kz/yfq5dclx7cd6vx_pj9dj5d6m0000gn/T/ipykernel_22347/3387183067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_group['completion_rate'] = completion_rate_test


,client_id,tenure_yr,tenure_mnth,age,gender,num_acc,balance,calls,log_ons,variation,visitor_id,visit_id,process_step,date_time,date,time,completion_rate
0,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:51:13,2017-04-02,11:51:13,0.692927
1,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:47:50,2017-04-02,11:47:50,0.692927
2,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:46:45,2017-04-02,11:46:45,0.692927
3,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,step_3,2017-04-02 11:23:08,2017-04-02,11:23:08,0.692927
4,836976,6.0,73.0,60.5,u,2.0,45105.3,6.0,9.0,test,427070339_1413275162,228976764_46825473280_96584,step_2,2017-04-02 11:22:24,2017-04-02,11:22:24,0.692927


#### Average Time Spent on Each Stage

In [6]:
# Sort the data by client_id and time
test_group.sort_values(by=['client_id', 'date_time'], inplace=True)

#create a column containing the date_time of the next row for comparison purposes
test_group['step_time'] = test_group.groupby('client_id')['date_time'].shift(-1)

#Convert the 'step_time' and 'date_time' columns to datetime datatype
test_group['step_time'] = pd.to_datetime(test_group['step_time'])
test_group['date_time'] = pd.to_datetime(test_group['date_time'])

#Calculate the difference between 'step_time' and 'date_time' to determine time spent on each step
test_group['time_spent'] = (test_group['step_time'] - test_group['date_time']).dt.total_seconds()

#Drop any missing values as confirm step will not have a comparison time
test_group = test_group.dropna(subset = 'time_spent')

#Group by process_step and calculate the average step duration in seconds
test_avg_step_dur = test_group.groupby('process_step')['time_spent'].mean().reset_index(name = 'average_step_duration_seconds')
test_avg_step_dur

/var/folders/kz/yfq5dclx7cd6vx_pj9dj5d6m0000gn/T/ipykernel_22347/614288555.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_group.sort_values(by=['client_id', 'date_time'], inplace=True)
/var/folders/kz/yfq5dclx7cd6vx_pj9dj5d6m0000gn/T/ipykernel_22347/614288555.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_group['step_time'] = test_group.groupby('client_id')['date_time'].shift(-1)
/var/folders/kz/yfq5dclx7cd6vx_pj9dj5d6m0000gn/T/ipykernel_22347/614288555.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,process_step,average_step_duration_seconds
0,confirm,356541.199328
1,start,101116.003966
2,step_1,43054.904265
3,step_2,21266.629605
4,step_3,33122.500283


There seems to be a large amounts of outliers in the dataset as some clients spent a long time on a single step. Lets use a boxplot to visualize this.

In [8]:
#Shape of distribution
skewness_price = test_group['time_spent'].skew()
kurtosis_price = test_group['time_spent'].kurtosis()

skewness_price, kurtosis_price

(8.696998611917905, 86.82521481236108)

1. **Skewness:** The positive value of skewness (8.69) for the 'time_spent' indicates that the distribution is right-skewed. Meaning that the tail on the right side is longer than the right side.
2. **Kurtosis:** The kurtosis value of 86.82 indicates the presence of a large number of outliers. This can explain why the average duration in seconds of each group seems very large. 

In [9]:
#Removing outliers in 'time_spent'
q1 = test_group['time_spent'].quantile(0.25)
q3 = test_group['time_spent'].quantile(0.75)
iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

#Filter out the outliers
test_group = test_group[(test_group['time_spent'] >= lower_bound) & (test_group['time_spent'] <= upper_bound)]

In [10]:
#Shape of distribution following removal of outliers
skewness_price_no_outliers = test_group['time_spent'].skew()
kurtosis_price_no_outliers = test_group['time_spent'].kurtosis()

skewness_price_no_outliers, kurtosis_price_no_outliers

(1.530269556881224, 1.949937697207735)

In [11]:
avg_step_duration_test = test_group.groupby('process_step')['time_spent'].mean().reset_index(name = 'average_step_duration_seconds')
avg_step_duration_test

,process_step,average_step_duration_seconds
0,confirm,77.179596
1,start,31.286976
2,step_1,39.568196
3,step_2,67.548211
4,step_3,65.540621


In [12]:
overall_avg_duration = test_group['time_spent'].mean()
overall_avg_duration

47.83179859761764

In [13]:
test_group = pd.merge(test_group, avg_step_duration_test, on = 'process_step', how = 'left')
test_group['avg_duration'] = overall_avg_duration
test_group.head()

,client_id,tenure_yr,tenure_mnth,age,gender,num_acc,balance,calls,log_ons,variation,...,visit_id,process_step,date_time,date,time,completion_rate,step_time,time_spent,average_step_duration_seconds,avg_duration
0,555,3.0,46.0,29.5,u,2.0,25454.66,2.0,6.0,test,...,637149525_38041617439_716659,start,2017-04-15 12:57:56,2017-04-15,12:57:56,0.692927,2017-04-15 12:58:03,7.0,31.286976,47.831799
1,555,3.0,46.0,29.5,u,2.0,25454.66,2.0,6.0,test,...,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,2017-04-15,12:58:03,0.692927,2017-04-15 12:58:35,32.0,39.568196,47.831799
2,555,3.0,46.0,29.5,u,2.0,25454.66,2.0,6.0,test,...,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,2017-04-15,12:58:35,0.692927,2017-04-15 13:00:14,99.0,67.548211,47.831799
3,555,3.0,46.0,29.5,u,2.0,25454.66,2.0,6.0,test,...,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,2017-04-15,13:00:14,0.692927,2017-04-15 13:00:34,20.0,65.540621,47.831799
4,647,12.0,151.0,57.5,m,2.0,30525.80,0.0,4.0,test,...,40369564_40101682850_311847,start,2017-04-12 15:41:28,2017-04-12,15:41:28,0.692927,2017-04-12 15:41:35,7.0,31.286976,47.831799


#### Error Rate

In [ ]:
#mapping of 'process_step' to determine instances of backward movements
step_mapping = {'start':0, 'step_1':1, 'step_2':2, 'step_3':3, 'confirm':4}
test_group['step_num'] = test_group['process_step'].map(step_mapping)

#create a column containing the step position of the next row for comparison purposes
test_group['next_step_position'] = test_group.groupby('client_id')['step_num'].shift(-1)

#create another column to check whether backward steps were taken
test_group['backward_step'] = test_group['step_num'] > test_group['next_step_position']

# Count occurrences of each step
step_count = test_group['process_step'].value_counts().reset_index()
step_count.columns = ['process_step', 'total_count']

#count occurences of backward step
backward_step_count = test_group[test_group['backward_step']].groupby('process_step')['backward_step'].count().reset_index()
backward_step_count.columns = ['process_step', 'backward_count']

#Merge the total step with backward step 
step_error_rate = pd.merge(step_count, backward_step_count, on='process_step', how='left')

#Fill the missing value with '0' to indicate no backward step
step_error_rate['backward_count'] = step_error_rate['backward_count'].fillna(0)

#calculate error rate of each step by dividing 'backward_count' by 'total_count'
step_error_rate['error_rate_by_step'] = step_error_rate['backward_count'] / step_error_rate['total_count']
step_error_rate

In [ ]:
errors_by_group_test = test_group['backward_step'].sum()
total_per_group_test = test_group['backward_step'].count()
error_rate_test = errors_by_group_test / total_per_group_test
error_rate_test

In [ ]:
test_group = pd.merge(test_group, step_error_rate, on = 'process_step', how = 'left')

In [ ]:
test_group['error_rate'] = error_rate_test

In [ ]:
drop_column = ['step_time', 'time_spent', 'step_num', 'next_step_position', 'backward_step', 'total_count', 'backward_count']
test_group = test_group.drop(columns = drop_column)
test_group.head(1)

### Lets explore the three KPIs for the 'Control' group

In [14]:
#Filter for 'Control' group
control_group = df[df['variation'] == 'control']

#### Completion Rate

In [15]:
#Completion rate of the test group
completion_control = control_group[control_group['process_step'] == 'confirm']
completion_control_unique = completion_control['client_id'].nunique()
total_control = control_group['client_id'].nunique()
completion_rate_control = completion_control_unique / total_control
print(f"The completion rate for the Control group is: {completion_rate_control:.3f}")

The completion rate for the Control group is: 0.656


In [16]:
control_group['completion_rate'] = completion_rate_control
control_group.head()

/var/folders/kz/yfq5dclx7cd6vx_pj9dj5d6m0000gn/T/ipykernel_22347/123581242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_group['completion_rate'] = completion_rate_control


,client_id,tenure_yr,tenure_mnth,age,gender,num_acc,balance,calls,log_ons,variation,visitor_id,visit_id,process_step,date_time,date,time,completion_rate
11,2304905,7.0,94.0,58.0,u,2.0,110860.3,6.0,9.0,control,336210275_56978226960,724930510_65323235593_692754,confirm,2017-05-21 11:07:16,2017-05-21,11:07:16,0.655785
12,2304905,7.0,94.0,58.0,u,2.0,110860.3,6.0,9.0,control,336210275_56978226960,724930510_65323235593_692754,step_3,2017-05-21 11:04:53,2017-05-21,11:04:53,0.655785
13,2304905,7.0,94.0,58.0,u,2.0,110860.3,6.0,9.0,control,336210275_56978226960,724930510_65323235593_692754,step_2,2017-05-21 11:04:04,2017-05-21,11:04:04,0.655785
14,2304905,7.0,94.0,58.0,u,2.0,110860.3,6.0,9.0,control,336210275_56978226960,724930510_65323235593_692754,step_1,2017-05-21 11:03:31,2017-05-21,11:03:31,0.655785
15,2304905,7.0,94.0,58.0,u,2.0,110860.3,6.0,9.0,control,336210275_56978226960,724930510_65323235593_692754,start,2017-05-21 11:03:25,2017-05-21,11:03:25,0.655785


#### Average Time Spent on Each Stage

In [17]:
# Sort the data by client_id and time
control_group.sort_values(by=['client_id', 'date_time'], inplace=True)

#create a column containing the date_time of the next row for comparison purposes
control_group['step_time'] = control_group.groupby('client_id')['date_time'].shift(-1)

#Convert the 'step_time' and 'date_time' columns to datetime datatype
control_group['step_time'] = pd.to_datetime(control_group['step_time'])
control_group['date_time'] = pd.to_datetime(control_group['date_time'])

#Calculate the difference between 'step_time' and 'date_time' to determine time spent on each step
control_group['time_spent'] = (control_group['step_time'] - control_group['date_time']).dt.total_seconds()

#Drop any missing values as confirm step will not have a comparison time
control_group = control_group.dropna(subset = 'time_spent')

#Group by process_step and calculate the average step duration in seconds
control_avg_step_dur = control_group.groupby('process_step')['time_spent'].mean().reset_index(name = 'average_step_duration_seconds')
control_avg_step_dur

/var/folders/kz/yfq5dclx7cd6vx_pj9dj5d6m0000gn/T/ipykernel_22347/3676598128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_group.sort_values(by=['client_id', 'date_time'], inplace=True)
/var/folders/kz/yfq5dclx7cd6vx_pj9dj5d6m0000gn/T/ipykernel_22347/3676598128.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_group['step_time'] = control_group.groupby('client_id')['date_time'].shift(-1)
/var/folders/kz/yfq5dclx7cd6vx_pj9dj5d6m0000gn/T/ipykernel_22347/3676598128.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,process_step,average_step_duration_seconds
0,confirm,333580.458151
1,start,129895.433093
2,step_1,57872.297814
3,step_2,34645.069197
4,step_3,60698.987631


There seems to be a large amounts of outliers in the dataset as some clients spent a long time on a single step. Lets use a boxplot to visualize this.

In [18]:
#Shape of distribution
skewness_price_control = control_group['time_spent'].skew()
kurtosis_price_control = control_group['time_spent'].kurtosis()

skewness_price_control, kurtosis_price_control

(7.935063270310005, 71.08987459373498)

1. **Skewness:** The positive value of skewness (7.94) for the 'time_spent' indicates that the distribution is right-skewed. Meaning that the tail on the right side is longer than the right side.
2. **Kurtosis:** The kurtosis value of 71.09 indicates the presence of a large number of outliers. This can explain why the average duration in seconds of each group seems very large. 

In [19]:
#Removing outliers in 'time_spent'
Q1 = control_group['time_spent'].quantile(0.25)
Q3 = control_group['time_spent'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

#Filter out the outliers
control_group = control_group[(control_group['time_spent'] >= lower) & (control_group['time_spent'] <= upper)]

In [20]:
#Shape of distribution
skewness_price_control_no_outliers = control_group['time_spent'].skew()
kurtosis_price_control_no_outliers = control_group['time_spent'].kurtosis()

skewness_price_control_no_outliers, kurtosis_price_control_no_outliers

(1.5212187445313743, 1.933454502297244)

In [21]:
avg_step_duration = control_group.groupby('process_step')['time_spent'].mean().reset_index(name = 'average_step_duration_seconds')
avg_step_duration

,process_step,average_step_duration_seconds
0,confirm,52.944513
1,start,34.697098
2,step_1,34.113661
3,step_2,72.037879
4,step_3,76.785064


In [22]:
overall_avg_duration_control = control_group['time_spent'].mean()
overall_avg_duration_control

50.58791575804012

In [23]:
control_group = pd.merge(control_group, avg_step_duration, on = 'process_step', how = 'left')
control_group['avg_duration'] = overall_avg_duration_control
control_group.head()

,client_id,tenure_yr,tenure_mnth,age,gender,num_acc,balance,calls,log_ons,variation,...,visit_id,process_step,date_time,date,time,completion_rate,step_time,time_spent,average_step_duration_seconds,avg_duration
0,1028,12.0,145.0,36.0,m,3.0,103520.22,1.0,4.0,control,...,557292053_87239438319_391157,start,2017-04-08 18:51:28,2017-04-08,18:51:28,0.655785,2017-04-08 18:52:17,49.0,34.697098,50.587916
1,1028,12.0,145.0,36.0,m,3.0,103520.22,1.0,4.0,control,...,557292053_87239438319_391157,step_1,2017-04-08 18:52:17,2017-04-08,18:52:17,0.655785,2017-04-08 18:53:20,63.0,34.113661,50.587916
2,1028,12.0,145.0,36.0,m,3.0,103520.22,1.0,4.0,control,...,557292053_87239438319_391157,step_1,2017-04-08 18:53:20,2017-04-08,18:53:20,0.655785,2017-04-08 18:53:29,9.0,34.113661,50.587916
3,1028,12.0,145.0,36.0,m,3.0,103520.22,1.0,4.0,control,...,557292053_87239438319_391157,step_3,2017-04-08 18:58:04,2017-04-08,18:58:04,0.655785,2017-04-08 18:59:55,111.0,76.785064,50.587916
4,1028,12.0,145.0,36.0,m,3.0,103520.22,1.0,4.0,control,...,557292053_87239438319_391157,step_1,2017-04-08 18:59:55,2017-04-08,18:59:55,0.655785,2017-04-08 19:00:15,20.0,34.113661,50.587916


#### Error Rate

In [ ]:
#mapping of 'process_step' to determine instances of backward movements
step_mapping = {'start':0, 'step_1':1, 'step_2':2, 'step_3':3, 'confirm':4}
control_group['step_num'] = control_group['process_step'].map(step_mapping)

#create a column containing the step position of the next row for comparison purposes
control_group['next_step_position'] = control_group.groupby('client_id')['step_num'].shift(-1)

#create another column to check whether backward steps were taken
control_group['backward_step'] = control_group['step_num'] > control_group['next_step_position']

# Count occurrences of each step
step_counts = control_group['process_step'].value_counts().reset_index()
step_counts.columns = ['process_step', 'total_count']

#count occurences of backward step
backward_step_counts = control_group[control_group['backward_step']].groupby('process_step')['backward_step'].count().reset_index()
backward_step_counts.columns = ['process_step', 'backward_count']

#Merge the total step with backward step 
step_error_rates = pd.merge(step_counts, backward_step_counts, on='process_step', how='left')

#Fill the missing value with '0' to indicate no backward step
step_error_rates['backward_count'] = step_error_rates['backward_count'].fillna(0)

#calculate error rate of each step by dividing 'backward_count' by 'total_count'
step_error_rates['error_rate_by_step'] = step_error_rates['backward_count'] / step_error_rate['total_count']
step_error_rates

In [ ]:
errors_by_group_control = control_group['backward_step'].sum()
total_per_group_control = control_group['backward_step'].count()
error_rate_control = errors_by_group_control / total_per_group_control
error_rate_control

In [ ]:
control_group = pd.merge(control_group, step_error_rates, on = 'process_step', how = 'left')

In [ ]:
control_group['error_rate'] = error_rate_control

In [ ]:
drop_column = ['step_time', 'time_spent', 'step_num', 'next_step_position', 'backward_step', 'total_count', 'backward_count']
control_group = control_group.drop(columns = drop_column)
control_group.head(1)

In [ ]:
tableau_df = pd.concat([test_group, control_group], ignore_index=True)
tableau_df.head()

In [ ]:
tableau_df.isna().sum()

In [ ]:
#Save the dataset to CSV format for tableau
tableau_df.to_csv("tableau.csv")